In [1]:
import pandas as pd
import numpy as np
import tweepy
import datetime
import pytz
import time

key is secret

In [2]:
# 取得したキーを格納
BT = "AAAAAAAAAAAAAAAAAAAAAEdLhgEAAAAAKyu0W6PIo7Zj%2FNB1BpINf%2Fcm6gE%3DT0EVYBss7PGh2rxZ70CXaOvKMrCifCwfVwsK37dXrcPSA851gg"
CK = "zScfmXxDONhXg9cIjp2SEk6m6"
CS = "fuAXsoZjTU5zvE1MkievSa7oITjF414wU6EGQUSgBwuTSfT44h"
AT = "1557281594955923456-ae2iV205fcP82YzSkCj3T9fPCQEEjh"
AS = "yOYPCy5G7ty8L6OikJWewpbSdEvGGbl1ETHgrRdw8ffGU"

In [3]:
api = tweepy.Client(bearer_token=BT, consumer_key=CK, consumer_secret=CS, access_token=AT, access_token_secret=AS, wait_on_rate_limit=True)

In [4]:
def utc2jst(timestamp_utc):
    datetime_utc = datetime.datetime.strptime(timestamp_utc + ".000+0000", "%Y-%m-%dT%H:%M:%SZ.%f%z")
    datetime_jst = datetime_utc.astimezone(datetime.timezone(datetime.timedelta(hours=+9)))
    timestamp_jst = datetime.datetime.strftime(datetime_jst, '%Y-%m-%dT%H:%M:%SZ')
    return timestamp_jst

def jst2utc(timestamp_jst):
    datatime_jst = datetime.datetime.strptime(timestamp_jst + ".000+0000", "%Y-%m-%dT%H:%M:%SZ.%f%z")
    datetime_utc = datatime_jst.astimezone(datetime.timezone(datetime.timedelta(hours=-9)))
    timestamp_utc = datetime.datetime.strftime(datetime_utc, '%Y-%m-%dT%H:%M:%SZ')
    return timestamp_utc

In [5]:
search_word = "雨 OR 強風 OR 台風 OR防風 OR 暴風"
start_time = "2019-10-05T00:00:00Z"
end_time = "2019-10-15T23:59:59Z"
item_number = 10
tweets = api.search_all_tweets(query=search_word, end_time=jst2utc(end_time), start_time=jst2utc(start_time), max_results=item_number)

In [171]:
hoax_tweets = []
counter = 0
for response in tweepy.Paginator(api.search_all_tweets,
                                query = search_word,
                                user_fields = ['username', 'public_metrics', 'description', 'location',],
                                tweet_fields = ['created_at', 'geo', 'public_metrics', 'text', 'source', 'attachments'],
                                media_fields = ['url', 'type', ],
                                expansions = ['author_id', 'attachments.media_keys'],
                                 end_time=jst2utc(end_time),
                                 start_time=jst2utc(start_time),
                                 max_results=10):
    time.sleep(1)
    hoax_tweets.append(response)
    counter+=1
    if counter >1:
        break

In [172]:
result = []
user_dict = {}
# Loop through each response object
for response in hoax_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username, 
                              'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'description': user.description,
                              'location': user.location
                             }
    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet
        result.append({'author_id': tweet.author_id, 
                       'username': author_info['username'],
                       'author_followers': author_info['followers'],
                       'author_tweets': author_info['tweets'],
                       'author_description': author_info['description'],
                       'author_location': author_info['location'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count']
                      })

# Change this list of dictionaries into a dataframe
df = pd.DataFrame(result)

In [173]:
df

,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count
0,1138362382122594304,cmaUyNAf6i7Jd2Z,1665,29209,病気の進行療養中。家族も誰いない身寄りなし孤独死。憲法改悪反対、絡んでくる人即ﾌﾞﾛｯｸ。\...,None,RT @shiikazuo: 台風１５号災害のさい、住宅被害認定については、わが党議員の質問...,2019-10-15 14:59:57+00:00,489,0,0,0
1,1137677758241230848,okomechan0708,70,47669,ファンマは🍧🍚 おこめのすきぴのこと よろしくお願いします,炊飯器,RT @rikako187: 雨ですね、\n少し肌寒い。\n\nこんな日はヨガの練習をします...,2019-10-15 14:59:54+00:00,15,0,0,0
2,827740314483904512,zephy_music,1080,65693,詞。音。絵。動画。感性に従うままに創作活動中。\nたまに飯テロ。\n割とマイペース。\n【n...,日本 北海道 札幌市,RT @shokora1210: 投稿から3ヶ月、2000再生ありがとうございます！\n留音...,2019-10-15 14:59:54+00:00,8,0,0,0
3,3169635931,akixwooxmonxxx,489,51472,ベースとうさぎ。北陸キュフラー赤族。だいたい赤い人。ライブ会場でだいたい虫みたいな動きをして...,None,RT @po__kaki__to: つけまつげを付けすぎた女性、強風で目を閉じる事ができなく...,2019-10-15 14:59:52+00:00,46862,0,0,0
4,866301618328698880,UnderDogDkk,402,24837,猫が大好き。歩く人。\nボーカリストではなくエンターテイナー。\n「かませ犬」as #大槻ケ...,愛知県,@snowlight0409 雨が降るから洗車をするのですよ。\n我が洗車の妙義を披露せしめたい,2019-10-15 14:59:51+00:00,0,1,0,0
5,900287386826883073,situcdbba,2015,75594,テトラポット登と夏井誠吾👓💍,感想まとめはモーメント,ただこれだけは言わせて！雨の痕の最後のおまけトラックはこの世に存在するタイトルコールの中でい...,2019-10-15 14:59:51+00:00,0,0,8,0
6,68691003,caatilyann,3,140,,None,RT @nhk_hokkaido: #台風１９号 道東荒れた天気 高潮に厳重警戒\n大型の台...,2019-10-15 14:59:51+00:00,29,0,0,0
7,2921562757,hiro28_rmok,519,21689,新日本プロレスの高橋ヒロム選手❤️❤️大大大好きです❤️❤️ ❤️❤️❤️❤️もっともっとも...,がんもどきーず 京都府,@ytpon10 幸せの雨降りましたね❤️☔️❤️,2019-10-15 14:59:50+00:00,0,1,3,0
8,2333495520,fuuuuurai,182,40976,〖ふうらい〗映画とごはんと恐竜が好き 20↑ ¦ K CCさくら i7 WT 呪 twst ...,ふざけ散らせ人生,RT @airi0218: 【コスプレ】アイドルマスターsideM\nWORLD TRE@S...,2019-10-15 14:59:49+00:00,34,0,0,0
9,1229834605,J3S3B33,415,9767,,None,RT @po__kaki__to: つけまつげを付けすぎた女性、強風で目を閉じる事ができなく...,2019-10-15 14:59:48+00:00,46862,0,0,0
